# Traning Agent with DDQN

## Step 1: Import the libraries

In [ ]:
import time
import random
import math
from collections import deque

import numpy as np
from IPython.display import clear_output
import matplotlib.pyplot as plt
import torch

from gym_unity.envs import UnityEnv

In [ ]:
from ddqn_agent import DDQNAgent
from model import DDQNLinear

## Step 2: Create our environment

Initialize the environment in the code cell below.


In [ ]:
ENV_PATH = '../unity_envs/PushBlock/'
ENV_NAME = 'Unity Environment'
env = UnityEnv(ENV_PATH + ENV_NAME, worker_id=0, multiagent=True, no_graphics=True)

In [ ]:
# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: ", device)

## Step 3: Viewing our Enviroment

In [ ]:
print("The size of state is: ", env.observation_space.shape[0])
print("No. of Actions: ", env.action_space)

## Step 4: Creating our Agent

In [ ]:
STATE_SIZE = env.observation_space.shape[0]
ACTION_SIZE = 7
GAMMA = 0.99                # discount factor
BUFFER_SIZE = int(1e6)      # replay buffer size
BATCH_SIZE = 256            # Update batch size
LR = 1e-3                   # learning rate 
TAU = 1e-3                  # for soft update of target parameters
UPDATE_EVERY = 20           # how often to update the network 

agent = DDQNAgent(STATE_SIZE, ACTION_SIZE, BUFFER_SIZE, BATCH_SIZE, GAMMA, LR, TAU, UPDATE_EVERY, device)

## Step 5: Creating Helper Function

### Eplision Helper Function

In [ ]:
EPS_START = 0.99       # starting value of epsilon
EPS_END = 0.01         # Ending value of epsilon
EPS_DECAY = 200        # Rate by which epsilon to be decayed

epsilon_by_epsiode = lambda frame_idx: EPS_END + (EPS_START - EPS_END) * math.exp(-1. * frame_idx / EPS_DECAY)

plt.plot([epsilon_by_epsiode(i) for i in range(2000)])
plt.show()

### Plot Traning

In [ ]:
def plot_result(scores):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.plot(np.arange(len(scores)), scores)
    plt.ylabel('Score')
    plt.xlabel('Episode #')
    plt.show()


## Step 6: Train the Agent with DQN

In [ ]:
def train(n_episodes, max_steps, scores_average_window, benchmark_reward):
    """
    Params
    ======
        n_episodes (int): maximum number of training episodes
        benchmark_reward (float): benchmark reward at which environment is solved.
    """
    scores = []
    scores_window = deque(maxlen=scores_average_window)
    for i_episode in range(1, n_episodes+1):
        states = np.array(env.reset())
        score = np.zeros(32)
        eps = epsilon_by_epsiode(i_episode)
        for step in range(max_steps):
            actions = agent.act(states, eps)
            next_states, rewards, dones, _ = env.step(actions.tolist())
            score += rewards
            agent.step(states, actions, rewards, next_states, dones)
            if np.any(dones):
                break
            else:
                states = np.array(next_states)
        scores_window.append(np.mean(score))       # save most recent score
        scores.append(np.mean(score))              # save most recent score
        
        # printing and ploting results
        clear_output(wait=True)
        plot_result(scores)
        print('\rEpisode {}\tAverage Score: {:.2f}\tEplision: {:.3f}'.format(i_episode, np.mean(scores_window), eps))
        
        if float(np.mean(scores_window)) >= benchmark_reward:
            agent.save_model("pushblock_solved.pth", scores)
            print("Yah Environment is solved :)")
            break
    
    return scores

In [ ]:
BENCHMARK_REWARD = 4.3
SCORES_AVERAGE_WINDOW = 100
NUM_EPISODES = 2000
MAX_STEPS = 3000

scores = train(NUM_EPISODES, MAX_STEPS, SCORES_AVERAGE_WINDOW, BENCHMARK_REWARD)
print("Done Training")

In [ ]:
env.close()